In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
import csv
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
import re
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from scipy.stats import loguniform
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix

In [37]:
path = "drive/MyDrive/Amazon ML Challenge/test.csv"
df_test = pd.read_csv(path, escapechar = "\\", quoting = csv.QUOTE_NONE)

In [38]:
path = "drive/MyDrive/Amazon ML Challenge/train.csv"
df_train = pd.read_csv(path, escapechar = "\\", quoting = csv.QUOTE_NONE)
df_train = df_train.dropna()

In [ ]:
punctuation_signs = list("?:!.,;")
nltk.download('punkt')
nltk.download('wordnet')
wordnet_lemmatizer = WordNetLemmatizer()
nltk.download('stopwords')
stop_words = list(stopwords.words('english'))

In [40]:
df_train['Title'] = df_train['TITLE'].str.replace("\r", " ")
df_train['Title'] = df_train['Title'].str.replace("\n", " ")
df_train['Title'] = df_train['Title'].str.replace("    ", " ")
df_train['Title'] = df_train['Title'].str.replace('"', '')
df_train['Title'] = df_train['Title'].str.lower()
for punct_sign in punctuation_signs:
  df_train['Title'] = df_train['Title'].str.replace(punct_sign, '')
df_train['Title'] = df_train['Title'].str.replace("'s", "") 

In [41]:
final_cols = ["Title", "BROWSE_NODE_ID"]
df_train = df_train[final_cols]
df_train = df_train.iloc[:200000, :]

In [42]:
df_test['Title'] = df_test['TITLE'].str.replace("\r", " ")
df_test['Title'] = df_test['Title'].str.replace("\n", " ")
df_test['Title'] = df_test['Title'].str.replace("    ", " ")
df_test['Title'] = df_test['Title'].str.replace('"', '')
df_test['Title'] = df_test['Title'].str.lower()
for punct_sign in punctuation_signs:
  df_test['Title'] = df_test['Title'].str.replace(punct_sign, '')
df_test['Title'] = df_test['Title'].str.replace("'s", "")

In [43]:
final_cols = ["Title", "PRODUCT_ID"]
df_test = df_test[final_cols]

In [45]:
X_train, X_test, y_train = df_train["Title"], df_test["Title"], df_train["BROWSE_NODE_ID"]

In [46]:
tfidf = TfidfVectorizer(stop_words='english', ngram_range=(1,2), min_df=5)
X_train_vectors_tfidf = tfidf.fit_transform(X_train)
X_test_vectors_tfidf = tfidf.transform(X_test)

In [ ]:
fig = px.bar(x = X_train_vectors_tfidf, y = y_train, log_y=True, color=[0.0]*len(class_distri[0])) # Logarithmically scaled because large imbalance.
fig.show()

# LGBM

In [22]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform as sp_uniform
from scipy.stats import randint as sp_randint

lgb = LGBMClassifier().fit(X_train_vectors_tfidf, y_train)

In [24]:
y_predict = lgb.predict(X_test_vectors_tfidf)

In [25]:
d = {"BROWSE_NODE_ID" : y_predict}
df1 = pd.DataFrame(data=d)

In [26]:
df_new = pd.concat([df_test, df1], axis = 1)
l = ["PRODUCT_ID", "BROWSE_NODE_ID"]
df_new = df_new[l]

In [27]:
df_new.to_csv("drive/MyDrive/Amazon ML Challenge/submission_LGBM.csv", index = False, header = True)

In [ ]:
path = "drive/MyDrive/Amazon ML Challenge/submission.csv"
df_sub = pd.read_csv(path)
df_sub.head()

# Logistic Regression


In [33]:
import gc
gc.collect()

876

In [ ]:
lr_tfidf = LogisticRegression(solver = 'liblinear', C=10, penalty = 'l2')
lr_tfidf.fit(X_train_vectors_tfidf, y_train)

In [ ]:
y_predict = lr_tfidf.predict(X_test_vectors_tfidf)

In [ ]:
d = {"BROWSE_NODE_ID" : y_predict}
df1 = pd.DataFrame(data=d)

In [ ]:
df_new = pd.concat([df_test, df1], axis = 1)

In [ ]:
l = ["PRODUCT_ID", "BROWSE_NODE_ID"]
df_new = df_new[l]

In [ ]:
df_new.to_csv("drive/MyDrive/Amazon ML Challenge/submission.csv", index = False, header = True)

In [ ]:
path = "drive/MyDrive/Amazon ML Challenge/submission.csv"
df_sub = pd.read_csv(path)
df_sub.head()